In [ ]:
Reproducibility

##Paper
# “Drosophila anti-nematode and antibacterial immune regulators revealed by RNA-Seq,” BMC Genomics 
# https://doi.org/10.1186/s12864-015-1690-2

In [ ]:
##Get Data

#RNA-seq Data
#https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?token-szetgcuwtvsvhyf&acc=GSE61466

# Reference Genome
#ftp://ftp.ensembl.org/pub/release-78/fasta/drosophila_melanogaster/dna/
wget ftp://ftp.ensembl.org/pub/release-78/fasta/drosophila_melanogaster/dna/ -r
    
# .gtf file
wget...

In [1]:
##Get Software
Bioconda
# Steps to set up bioconda
# 1. Download miniconda
wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
chmod +x Miniconda3-latest-Linux-x86_64.sh
./Miniconda3-latest-Linux-x86_64.sh

# 2. log out the server
# 3. set up channels
conda config --add channels defaults
conda config --add channels bioconda
conda config --add channels conda-forge

## to install packages
conda install r

## to search for packages
conda search bwa

## Reference: https://bioconda.github.io

# SRAtoolkit
conda install sra-tools

FastQC
Bowtie
Tophat
HTseq
R conda install r
DEseq conda install bioconductor-deseq
Sam tools


SyntaxError: invalid syntax (<ipython-input-1-cbb99279c00c>, line 10)